In [5]:
pip install tweepy pandas python-dateutil


   ---------------------------------------- 0/3 [oauthlib]
   ---------------------------------------- 0/3 [oauthlib]
   ---------------------------------------- 0/3 [oauthlib]
   ---------------------------------------- 0/3 [oauthlib]
   ---------------------------------------- 0/3 [oauthlib]
   -------------------------- ------------- 2/3 [tweepy]
   -------------------------- ------------- 2/3 [tweepy]
   -------------------------- ------------- 2/3 [tweepy]
   ---------------------------------------- 3/3 [tweepy]

Note: you may need to restart the kernel to use updated packages.


In [8]:
import os, json, time, math, sys
from datetime import datetime, timedelta, timezone
from typing import Dict, Iterable, Optional, List

import tweepy
import pandas as pd

In [26]:
# 검색어(필요시 키워드 조정): 한국어만, 리트윗 제외
QUERY = '(경제뉴스 OR "경제 뉴스" OR "경제 기사") lang:ko -is:retweet'

# 기간: 최근 12시간(끝 시각은 현재보다 60초 이전으로 안전 설정)
END_TIME   = datetime.now(timezone.utc) - timedelta(seconds=60)
START_TIME = END_TIME - timedelta(days=3)

# 요청량 보수적으로
MAX_TWEETS_TOTAL = 200    # 총 상한
PAGE_SIZE        = 50     # 페이지당 개수 (최대 100이지만 50으로 여유)
PAGE_SLEEP_SEC   = 1.0    # 각 페이지 사이 1초 슬립(레이트리밋 완화)

In [28]:
# ---------- 1) 상태 출력 ----------
KST = timezone(timedelta(hours=9))
print("⏱️  Now(UTC):", datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S"))
print("⏱️  Now(KST):", datetime.now(KST).strftime("%Y-%m-%d %H:%M:%S"))
print("📅  Window(UTC):", START_TIME.strftime("%Y-%m-%d %H:%M:%S"), "→", END_TIME.strftime("%Y-%m-%d %H:%M:%S"))
print(f"⚙️  Limits: MAX_TWEETS_TOTAL={MAX_TWEETS_TOTAL}, PAGE_SIZE={PAGE_SIZE}, PAGE_SLEEP_SEC={PAGE_SLEEP_SEC}")

# ---------- 2) 클라이언트 ----------
client = tweepy.Client(bearer_token=BEARER_TOKEN, wait_on_rate_limit=False)

# ---------- 3) 유틸: 파일/재개 ----------
def iter_jsonl(path: str):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        return
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                yield json.loads(line)
            except json.JSONDecodeError:
                continue

def load_existing_ids(path: str) -> (set, Optional[int]):
    """기존 JSONL에서 ID 집합과 최대 ID(since_id) 반환"""
    seen, max_id = set(), None
    for obj in iter_jsonl(path):
        tid = obj.get("id")
        if tid is None:
            continue
        tid = int(tid)
        seen.add(tid)
        if max_id is None or tid > max_id:
            max_id = tid
    return seen, max_id

def append_jsonl(path: str, rows: List[Dict]):
    if not rows:
        return
    with open(path, "a", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

def append_csv(path: str, rows: List[Dict]):
    if not rows:
        return
    df = pd.DataFrame(rows)
    header = not os.path.exists(path) or os.path.getsize(path) == 0
    df.to_csv(path, index=False, mode="a", header=header)

# ---------- 4) 1회 프로브(상태 점검용) ----------
def one_shot_probe() -> bool:
    """토큰/쿼리/한도 상태를 한 번 가볍게 점검"""
    try:
        resp = client.search_recent_tweets(
            query=QUERY,
            max_results=10,
            tweet_fields=["created_at"]
        )
        cnt = 0 if not resp.data else len(resp.data)
        print(f"✅ Probe OK — got {cnt} tweets in this window")
        if cnt:
            print("  sample:", resp.data[0].text[:80].replace("\n"," "))
        return True
    except tweepy.Unauthorized:
        print("❌ 401 Unauthorized — Bearer Token 확인 필요")
        return False
    except tweepy.TooManyRequests as e:
        headers = getattr(e.response, "headers", {})
        reset = headers.get("x-rate-limit-reset")
        print("⚠️ 429 TooManyRequests (probe)")
        if reset:
            reset_dt_utc = datetime.fromtimestamp(int(reset), tz=timezone.utc)
            print("  reset(UTC):", reset_dt_utc.strftime("%Y-%m-%d %H:%M:%S"),
                  "| reset(KST):", reset_dt_utc.astimezone(KST).strftime("%Y-%m-%d %H:%M:%S"))
        else:
            print("  reset 헤더 없음 — 잠시 후 재시도 권장")
        return False
    except Exception as e:
        print("❌ Unexpected error in probe:", repr(e))
        return False

probe_ok = one_shot_probe()

# ---------- 5) 수집 ----------
def polite_sleep_until(reset_ts: int):
    """x-rate-limit-reset까지 1초 단위로 카운트다운(언제든 Interrupt 가능)"""
    target = datetime.fromtimestamp(reset_ts, tz=timezone.utc)
    while True:
        now = datetime.now(timezone.utc)
        left = (target - now).total_seconds()
        if left <= 0:
            break
        to_sleep = 1 if left > 1 else left
        print(f"\r⏳ rate limit… 남은 {int(left)}초", end="")
        time.sleep(to_sleep)
    print("\r✅ 재시도 시작                                                ")

def collect_recent_safely():
    seen, since_id = load_existing_ids(OUTPUT_JSONL)
    print(f"📂 Resume: existing={len(seen)} rows, since_id={since_id}")

    collected = 0
    buf_jsonl: List[Dict] = []
    buf_csv:   List[Dict] = []
    next_token = None

    try:
        while collected < MAX_TWEETS_TOTAL:
            try:
                resp = client.search_recent_tweets(
                    query=QUERY,
                    start_time=START_TIME.isoformat(),
                    end_time=END_TIME.isoformat(),
                    max_results=PAGE_SIZE,
                    tweet_fields=["created_at","lang","public_metrics","source"],
                    expansions=["author_id"],
                    user_fields=["username","public_metrics"],
                    next_token=next_token,
                    since_id=since_id
                )
            except tweepy.TooManyRequests as e:
                print("\n⚠️ 429 TooManyRequests")
                headers = getattr(e.response, "headers", {})
                reset = headers.get("x-rate-limit-reset")
                if reset:
                    reset_dt_utc = datetime.fromtimestamp(int(reset), tz=timezone.utc)
                    print("   reset(UTC):", reset_dt_utc.strftime("%Y-%m-%d %H:%M:%S"),
                          "| reset(KST):", reset_dt_utc.astimezone(KST).strftime("%Y-%m-%d %H:%M:%S"))
                    polite_sleep_until(int(reset))
                    continue
                else:
                    # 헤더가 없으면 60초만 보수적으로 대기
                    for i in range(60, 0, -1):
                        print(f"\r⏳ 60초 대기… {i}s", end="")
                        time.sleep(1)
                    print("\r✅ 재시도 시작                            ")
                    continue
            except KeyboardInterrupt:
                print("\n🛑 사용자 중단 (Interrupt)")
                break
            except Exception as e:
                print("❌ Unexpected error:", repr(e))
                break

            if not resp or not resp.data:
                print("ℹ️ 더 이상 결과 없음(이 기간/쿼리).")
                break

            users = {u.id: u for u in (resp.includes.get("users", []) if resp.includes else [])}
            new_rows = []
            for t in resp.data:
                tid = int(t.id)
                if tid in seen:
                    continue
                seen.add(tid)
                u = users.get(t.author_id)
                new_rows.append({
                    "id": tid,
                    "date": t.created_at.isoformat() if t.created_at else None,
                    "text": t.text,
                    "username": getattr(u, "username", None),
                    "followers": (getattr(u, "public_metrics", {}) or {}).get("followers_count"),
                    "likes": t.public_metrics.get("like_count", 0) if t.public_metrics else 0,
                    "retweets": t.public_metrics.get("retweet_count", 0) if t.public_metrics else 0,
                    "replies": t.public_metrics.get("reply_count", 0) if t.public_metrics else 0,
                    "source": t.source
                })

            if new_rows:
                buf_jsonl.extend(new_rows)
                buf_csv.extend(new_rows)
                collected += len(new_rows)
                print(f"📥 이번 페이지 신규: {len(new_rows)}건 | 누적: {collected}/{MAX_TWEETS_TOTAL}")

            # 배치 저장
            if len(buf_jsonl) >= 100:
                append_jsonl(OUTPUT_JSONL, buf_jsonl)
                append_csv(OUTPUT_CSV, buf_csv)
                print(f"💾 저장 +{len(buf_jsonl)} (total so far={collected})")
                buf_jsonl.clear(); buf_csv.clear()

            # 상한 도달 시 종료
            if collected >= MAX_TWEETS_TOTAL:
                print(f"✅ 상한 도달: {MAX_TWEETS_TOTAL}")
                break

            next_token = resp.meta.get("next_token")
            if not next_token:
                print("✅ 더 가져올 페이지가 없습니다.")
                break

            # 페이지 사이 짧은 슬립(레이트리밋 완화 + 인터럽트 친화)
            time.sleep(PAGE_SLEEP_SEC)

    except KeyboardInterrupt:
        print("\n🛑 사용자 중단 (try/except)")

    finally:
        # 남은 버퍼 저장
        if buf_jsonl:
            append_jsonl(OUTPUT_JSONL, buf_jsonl)
            append_csv(OUTPUT_CSV, buf_csv)
            print(f"💾 최종 저장 +{len(buf_jsonl)}")
        print(f"🎉 종료. 이번 실행에서 신규 수집: {collected}건")

⏱️  Now(UTC): 2025-10-14 11:18:56
⏱️  Now(KST): 2025-10-14 20:18:56
📅  Window(UTC): 2025-10-11 11:01:08 → 2025-10-14 11:01:08
⚙️  Limits: MAX_TWEETS_TOTAL=200, PAGE_SIZE=50, PAGE_SLEEP_SEC=1.0
✅ Probe OK — got 10 tweets in this window
  sample: 경제 뉴스 제공 thông tin về tình hình tăng trưởng, التضخم والاستثمار والتجارة.


---

In [10]:
# 검색어: OR로 확장 가능 (리트윗 제외, 한국어만)
QUERY = '("경제 뉴스 어렵다" OR "경제 기사 어렵다" OR "경제 용어 어렵다") lang:ko -is:retweet'

# 기간: 최근 7일 (end_time은 현재보다 60초 이전으로 안전 설정)
END_TIME   = datetime.now(timezone.utc) - timedelta(seconds=60)
START_TIME = END_TIME - timedelta(days=7)

# 수집 한도(너무 오래 돌지 않도록 안전 상한)
MAX_TWEETS_TOTAL = 1000     # 원하는 총 수집 개수
PAGE_SIZE        = 100      # API 허용 최대 100
SHORT_SLEEP_SEC  = 1        # 레이트리밋 시 1초 단위로 짧게 대기(인터럽트 친화)

# ---------- 1) 클라이언트 ----------
client = tweepy.Client(bearer_token=BEARER_TOKEN, wait_on_rate_limit=False)

In [11]:
# ---------- 2) 유틸: 파일/재개 ----------
def iter_jsonl(path: str):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        return
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                yield json.loads(line)
            except json.JSONDecodeError:
                continue

def load_existing_ids(path: str) -> (set, Optional[int]):
    """기존 JSONL에서 ID 집합과 최대 ID(since_id) 반환"""
    seen, max_id = set(), None
    for obj in iter_jsonl(path):
        tid = obj.get("id")
        if tid is None:
            continue
        seen.add(int(tid))
        if max_id is None or int(tid) > max_id:
            max_id = int(tid)
    return seen, max_id

def append_jsonl(path: str, rows: List[Dict]):
    if not rows:
        return
    with open(path, "a", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

def append_csv(path: str, rows: List[Dict]):
    if not rows:
        return
    df = pd.DataFrame(rows)
    header = not os.path.exists(path) or os.path.getsize(path) == 0
    df.to_csv(path, index=False, mode="a", header=header)

In [12]:
# ---------- 3) 수집 제너레이터 ----------
def search_recent_generator(
    query: str,
    start_time: datetime,
    end_time: datetime,
    page_size: int = 100,
    since_id: Optional[int] = None,
) -> Iterable[Dict]:
    """
    Tweepy v2 search_recent_tweets 페이징 제너레이터
    - 레이트리밋 시 TooManyRequests 캐치 → 1초 대기 반복 (언제든 Interrupt 가능)
    - 각 배치 후 즉시 yield
    """
    next_token = None
    while True:
        try:
            resp = client.search_recent_tweets(
                query=query,
                start_time=start_time.isoformat(),
                end_time=end_time.isoformat(),
                max_results=page_size,
                tweet_fields=["created_at", "lang", "public_metrics", "source", "context_annotations", "in_reply_to_user_id"],
                expansions=["author_id"],
                user_fields=["username", "name", "public_metrics"],
                next_token=next_token,
                since_id=since_id
            )
        except tweepy.TooManyRequests as e:
            # 레이트리밋 → 1초 단위로 짧게 대기하여 언제든 Interrupt 가능
            # (남은 시간 정보가 없으므로 짧은 루프 대기)
            print("⚠️ Rate limit hit → short-sleep loop… (Ctrl+C로 언제든 중단 가능)")
            try:
                # 최대 15분까지 대기(짧게 끊어서)
                wait_cap = 15 * 60
                waited = 0
                while waited < wait_cap:
                    time.sleep(SHORT_SLEEP_SEC)
                    waited += SHORT_SLEEP_SEC
            except KeyboardInterrupt:
                print("🛑 사용자 중단(KeyboardInterrupt) → 지금까지 수집분만 반환")
                return
            else:
                # 대기 후 재시도
                continue
        except KeyboardInterrupt:
            print("🛑 사용자 중단(KeyboardInterrupt) → 지금까지 수집분만 반환")
            return
        except Exception as e:
            print(f"❌ Unexpected error: {repr(e)}")
            return

        if not resp.data:
            return

        users = {u.id: u for u in (resp.includes.get("users", []) if resp.includes else [])}
        batch = []
        for t in resp.data:
            u = users.get(t.author_id)
            item = {
                "id": int(t.id),
                "date": t.created_at.isoformat() if t.created_at else None,
                "lang": t.lang,
                "text": t.text,
                "author_id": t.author_id,
                "username": getattr(u, "username", None),
                "name": getattr(u, "name", None),
                "followers": (getattr(u, "public_metrics", {}) or {}).get("followers_count"),
                "retweets": t.public_metrics.get("retweet_count", 0) if t.public_metrics else None,
                "replies":  t.public_metrics.get("reply_count", 0) if t.public_metrics else None,
                "likes":    t.public_metrics.get("like_count", 0) if t.public_metrics else None,
                "quotes":   t.public_metrics.get("quote_count", 0) if t.public_metrics else None,
                "source":   t.source,
            }
            batch.append(item)

        for row in batch:
            yield row

        next_token = resp.meta.get("next_token")
        if not next_token:
            return

In [13]:
# ---------- 4) 실행(체크포인트/재개/중복제거/증분저장) ----------
def run_collect():
    # 기존 파일에서 seen/ since_id 로드
    seen, since_id = load_existing_ids(OUTPUT_JSONL)
    print(f"📂 resume: existing={len(seen)} rows, since_id={since_id}")

    collected = 0
    buffer_jsonl: List[Dict] = []
    buffer_csv:   List[Dict] = []

    try:
        for row in search_recent_generator(
            query=QUERY,
            start_time=START_TIME,
            end_time=END_TIME,
            page_size=PAGE_SIZE,
            since_id=since_id,
        ):
            tid = row["id"]
            if tid in seen:
                continue
            seen.add(tid)
            buffer_jsonl.append(row)
            buffer_csv.append(row)
            collected += 1

            # ---- 배치 저장(메모리 과점 방지 + 체크포인트) ----
            if len(buffer_jsonl) >= 200:
                append_jsonl(OUTPUT_JSONL, buffer_jsonl)
                append_csv(OUTPUT_CSV, buffer_csv)
                print(f"💾 saved +{len(buffer_jsonl)} (total={collected})")
                buffer_jsonl.clear()
                buffer_csv.clear()

            # ---- 수집 상한 ----
            if collected >= MAX_TWEETS_TOTAL:
                print(f"✅ reached MAX_TWEETS_TOTAL={MAX_TWEETS_TOTAL}")
                break

    except KeyboardInterrupt:
        print("🛑 사용자 중단 → 부분 저장 실행")

    finally:
        # 남은 버퍼 저장
        if buffer_jsonl:
            append_jsonl(OUTPUT_JSONL, buffer_jsonl)
            append_csv(OUTPUT_CSV, buffer_csv)
            print(f"💾 saved (final flush) +{len(buffer_jsonl)}")
        print(f"🎉 done. total collected in this run: {collected}")

run_collect()

📂 resume: existing=0 rows, since_id=None
⚠️ Rate limit hit → short-sleep loop… (Ctrl+C로 언제든 중단 가능)
❌ Unexpected error: BadRequest("400 Bad Request\nInvalid 'start_time':'2025-10-07T10:27Z'. 'start_time' must be on or after 2025-10-07T10:42Z")
🎉 done. total collected in this run: 0


In [14]:
# ---------- 5) 결과 미리보기 ----------
try:
    df = pd.read_json(OUTPUT_JSONL, lines=True)
    print("총 행:", len(df))
    display(df.head(5)[["date","username","text"]])
except Exception as e:
    print("미리보기 실패:", e)

미리보기 실패: Unexpected character found when decoding 'true'


C:\Users\ysu13\AppData\Local\Temp\ipykernel_23532\459154607.py:3: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(OUTPUT_JSONL, lines=True)


In [15]:
import tweepy, os, time
from datetime import datetime, timezone

BEARER_TOKEN = BEARER_TOKEN  # 기존 변수 재사용

client = tweepy.Client(bearer_token=BEARER_TOKEN, wait_on_rate_limit=False)

def one_shot_probe():
    try:
        # 가장 가벼운 요청: 최근 검색 1페이지(10개), end/start 생략
        resp = client.search_recent_tweets(
            query='("경제 뉴스 어렵다" OR "경제 기사 어렵다") lang:ko -is:retweet',
            max_results=10,
            tweet_fields=["created_at"]
        )
        # Tweepy는 기본적으로 헤더를 안 내보내지만, 에러 시엔 예외로 확인
        print("✅ probe status: OK")
        print("data count:", 0 if not resp.data else len(resp.data))
        if resp.data:
            print("sample:", resp.data[0].text[:80])
        return True
    except tweepy.Unauthorized as e:
        print("❌ 401 Unauthorized → Bearer Token 확인 필요")
        return False
    except tweepy.TooManyRequests as e:
        # 남은 시간 계산(가능하면 헤더에서 reset 읽기)
        headers = getattr(e.response, "headers", {})
        reset = headers.get("x-rate-limit-reset")
        print("⚠️ 429 TooManyRequests")
        if reset:
            reset_dt = datetime.fromtimestamp(int(reset), tz=timezone.utc)
            wait_sec = (reset_dt - datetime.now(timezone.utc)).total_seconds()
            print(f" → 제한 해제 예상(UTC): {reset_dt} (약 {int(wait_sec)}초 후)")
        else:
            print(" → reset 헤더 없음: 몇 분 뒤 재시도")
        return False
    except tweepy.BadRequest as e:
        print("❌ 400 BadRequest → 시간 파라미터 등 확인 필요")
        print(e)
        return False
    except Exception as e:
        print("❌ Unexpected:", repr(e))
        return False

one_shot_probe()


⚠️ 429 TooManyRequests
 → 제한 해제 예상(UTC): 2025-10-14 10:58:07+00:00 (약 876초 후)


False